In [1]:
from omegaconf import OmegaConf as om
from omegaconf import DictConfig
from typing import cast
import sys 
import os
sys.path.append(os.path.abspath(".."))

#from main import build_model
import src.create_bert as bert_module
import src.create_model as model_module
from transformers import AutoTokenizer, Trainer, TrainingArguments



yaml_path = "../yamls/pretrain/embed_dna_monarch-mixer-pretrain-786dim-80m-parameters.yaml"

tokenizer = AutoTokenizer.from_pretrained("gagneurlab/SpeciesLM", revision="downstream_species_lm")


with open(yaml_path) as f:
    cfg = om.load(f)
cfg = cast(DictConfig, cfg)
print(cfg.max_duration)
model = model_module.create_model(cfg.model.get("model_config"))
#model = bert_module.create_bert_mlm(model_config = cfg.model.get("model_config", None))

Could not import FlashFFTConv!


/home/lukas/.conda/envs/m2_bert/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


200000ba
Using Monarch Mixer for Sequence Mixing: True
-- Bidirectional: True
-- Using Long Conv Residual: True
-- Hyena w: 10
-- Hyena w mod: 1
-- Hyena filter order: 128
-- Hyena filter dropout: 0.2
-- Hyena filter wd: 0.1
-- Hyena filter emb dim: 5
-- Hyena filter lr: 0.001
-- Hyena filter lr pos emb: 1e-05


In [2]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(5504, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (species_embeddings): Embedding(4096, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): MonarchMixerSequenceMixing(
            (filter_fn): HyenaFilter(
              (dropout): Dropout(p=0.2, inplace=False)
              (pos_emb): PositionalEmbedding()
              (implicit_filter): Sequential(
                (0): Linear(in_features=5, out_features=128, bias=True)
                (1): Sin()
                (2): Linear(in_features=128, out_features=128, bias=True)
                (3): Sin()
                (4): Linear(in_features=128, out_features=128, bias=Tru

In [5]:
from datasets import load_from_disk
from collate import DataCollatorForLanguageModelingSpan
from torch.utils.data import DataLoader

#Load model directly
%env WANDB_PROJECT=singlesamplednam2



dataset = load_from_disk("../batch_species")
dataset = dataset.remove_columns(["species_name", "__index_level_0__"])

data_collator = DataCollatorForLanguageModelingSpan(tokenizer, mlm=True, mlm_probability = 0.02, span_length = 6)

dataloader = DataLoader(dataset["train"], batch_size=1, collate_fn=data_collator)

sample = next(iter(dataloader))
#training_args = TrainingArguments(
#    output_dir="./results/correct_model_1sample",
#    
#    max_steps=100000,
#    
#    seed=17,
#    per_device_train_batch_size=1,
#    gradient_accumulation_steps=1,
#    
#    logging_strategy="steps",
#    logging_steps=1,
#    
#    remove_unused_columns=False,
#    evaluation_strategy="no",
#    
#    #dataloader_num_workers=0,
#    run_name="repo_model_1sample",
#    report_to="wandb"
#)
#
#trainer = Trainer(
#    model=model,
#    args=training_args,
#    train_dataset=dataset["train"],
#    eval_dataset=dataset["test"],
#    data_collator=data_collator
#)
#
#
##trainer.train()
#tokenizer.vocab_size

print(sample['species_id'].shape)
print(sample['input_ids'].shape)

env: WANDB_PROJECT=singlesamplednam2
torch.Size([1, 298])
torch.Size([1, 300])


In [10]:
model(**sample)

MaskedLMOutput(loss=tensor(8.7274, grad_fn=<NllLossBackward0>), logits=tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [5]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset["train"], batch_size=1, shuffle=True, collate_fn=data_collator)
for step, inputs in enumerate(train_dataloader):
    inputs = {
        "args": inputs
    }
    model(**inputs)

TypeError: BertForMaskedLM.forward() got an unexpected keyword argument 'args'